# Titanic - Machine Learning from Disaster

part 2 - model training

https://www.ahmedbesbes.com/blog/kaggle-titanic-competition 

https://towardsdatascience.com/explaining-feature-importance-by-example-of-a-random-forest-d9166011959e

https://towardsdatascience.com/kaggle-titanic-competition-model-building-tuning-in-python-12f4f74436b5

In [ ]:
%matplotlib inline

#load packages
import time
import joblib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from scipy.stats import spearmanr
from scipy.cluster import hierarchy

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.base import clone 
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline #need to specify names
from sklearn.pipeline import make_pipeline #auto generates names
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, cross_val_predict

In [ ]:
#get data
df = pd.read_csv('Data/clean.csv')

df_train = df.dropna(subset=['Survived'])
df_train = df_train.astype({'Survived': np.int})

df_test = df[df.isnull().any(axis=1)]
df_test = df_test.drop(['Survived'], axis=1)

## Correlation study

In [ ]:
colormap = plt.cm.coolwarm
plt.figure(figsize=(25,15))
sns.heatmap(df_train.corr().round(1), cmap=colormap, annot=True, linewidths=0.1)

### Univariate Analysis

In [ ]:
#plot ratio survivers
df_train['Survived'].value_counts().plot.pie(autopct='%.1f%%', explode=[0, 0.05])

### Bivariate Analysis

In [ ]:
#group by class and survived
df_train.groupby(['Pclass', 'Survived'])['Survived'].count() #used to count non-NA/null across axis

In [ ]:
sns.catplot(x='Pclass', y='Survived', data=df_train, kind='point')

In [ ]:
#chance of surviving by Fare
df_train[['CategoricalFare', 'Survived']].groupby('CategoricalFare').mean()

In [ ]:
df_train[['CategoricalFare', 'Survived']].groupby('CategoricalFare')['Survived'].mean().plot(kind='bar', figsize=(15, 7))

In [ ]:
#chance of surviving by Age
df_train[['CategoricalAge', 'Survived']].groupby('CategoricalAge').mean()

In [ ]:
df_train[['CategoricalAge', 'Survived']].groupby('CategoricalAge')['Survived'].mean().plot(kind='bar', figsize=(15, 7))

In [ ]:
fig, age_plot = plt.subplots(figsize = (10,5))
age_plot = sns.histplot(data=df_train, x="Age", hue="Survived", multiple="dodge", shrink=.8, kde=True)

In [ ]:
#relation between age and fare of passengers
sns.jointplot(x='Age', y='Fare', data=df_train, kind='hex')

In [ ]:
#relation between sex and fare of passengers
sns.catplot(x='Sex', y='Fare', data=df_train, kind='boxen')

In [ ]:
#relation between age and sex of passengers
sns.catplot(x='Sex', y='Age', data=df_train)

In [ ]:
#survival rate per deck
df_train[['Deck', 'Survived']].groupby('Deck')['Survived'].mean().plot(kind='bar', figsize=(15, 7))

In [ ]:
#chance of surviving by ticket frequency, identical tickets indicate people travel together
df_train[['TicketFrequency', 'Survived']].groupby('TicketFrequency').mean()

In [ ]:
#survival rater per family size
df_train[['FamilySizeBin', 'Survived']].groupby('FamilySizeBin')['Survived'].mean().plot(kind='bar', figsize=(15, 7))

Multivariate Analysis

In [ ]:
#relation between age, sex and class
sns.catplot(x='Sex', y='Age', data=df_train, kind='box', hue='Pclass')

In [ ]:
sns.catplot(x='Pclass', y='Age', data=df_train, kind='violin', hue='Sex')

In [ ]:
#relation between age, fare, sex and class
sns.relplot(x='Age', y='Fare', data=df_train, row='Sex', col='Pclass')

In [ ]:
#make a seperate plot dataset
df_plot = df_train.copy()

df_plot["Survived"] = df_plot["Survived"].replace({0:"No", 1:"Yes"}) 
df_plot["Pclass"] = df_plot["Pclass"].replace({1:"1st", 2:"2nd", 3:"3rd"}) 
df_plot["Embarked"] = df_plot["Embarked"].replace({1:"Cherbourg", 2:"Queenstown", 0:"Southampton"}) 
df_plot["Person"] = df_plot["Person"].replace({0:"Female", 1:"Male", 2:"Child"}) 

df_plot['SibSp'] = df_plot['SibSp'].astype(str)
df_plot['Parch'] = df_plot['Parch'].astype(str)

#create a subplot
fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(15,7))

sns.histplot(data=df_plot, x="Survived", hue="Embarked", multiple="dodge", shrink=.8, palette='bright', ax=axs[0,0])
sns.histplot(data=df_plot, x="Survived", hue="Pclass", multiple="dodge", shrink=.8, palette='bright', ax=axs[0,1])
sns.histplot(data=df_plot, x="Survived", hue="Person", multiple="dodge", shrink=.8, palette='bright', ax=axs[0,2])
sns.histplot(data=df_plot, x="SibSp", hue="Survived", multiple="dodge", shrink=.8, palette='bright', ax=axs[1,0])
sns.histplot(data=df_plot, x="Parch", hue="Survived", multiple="dodge", shrink=.8, palette='bright', ax=axs[1,1])
sns.histplot(data=df_plot, x="Fare", hue="Survived", multiple="dodge", shrink=.8, bins=10, palette='bright', ax=axs[1,2])

## Model training

In [ ]:
X_train = df_train[df_train.columns.difference(['Survived'], sort=False)].values
y_train = df_train[['Survived']].values.ravel()

### Test some different models

In [ ]:
def model_comparison(models, X_train, y_train):
    model_stats = {}
    
    for mname, minst in models.items():
        model_scores = []

        #create pipeline with scaler
        model_pipe = make_pipeline(StandardScaler(), minst)

        #fit model and save scores
        model_pipe.fit(X_train, y_train)
        model_scores.append(model_pipe.score(X_train, y_train))

        #implement a cross validation score
        scores = cross_val_score(model_pipe, X_train, y_train, cv=10, scoring='accuracy')
        model_scores.append(scores.mean())
        model_scores.append(scores.std())

        #implement cross validation predictions
        y_train_pred = cross_val_predict(model_pipe, X_train, y_train, cv=10)

        #calculate precision and recall
        precision = precision_score(y_train, y_train_pred)
        recall = recall_score(y_train, y_train_pred)
        model_scores.append(precision)
        model_scores.append(recall)

        #calculate F1 score
        f1 = 2*(precision*recall)/(precision+recall)
        model_scores.append(f1)

        #calculate ROC AUC score
        roc_auc = cross_val_score(model_pipe, X_train, y_train, cv=10, scoring='roc_auc').mean()
        model_scores.append(roc_auc)

        #create dictionary
        model_stats[mname] = model_scores

    colnames = ['accuracy','accuracy_cv_score','accuracy_cv_stddev',
                'precision_score','recall_score','f1_score', 'roc_auc__cv_score']

    df_model_stats = pd.DataFrame.from_dict(model_stats, orient='index', columns=colnames)
    df_model_stats_ranked = df_model_stats.sort_values(by='accuracy_cv_score', ascending=False)

    return df_model_stats_ranked

In [ ]:
models = {'RandomForestClassifier': RandomForestClassifier(random_state=42),
          'LogisticRegression': LogisticRegression(random_state=42),
          'SupportVectorClassifier': SVC(random_state=42),
          'DecisionTreeClassifier': DecisionTreeClassifier(random_state=42),
          'KNeighborsClassifier': KNeighborsClassifier(),
          'GaussianNaiveBayes': GaussianNB(),
          'Perceptron': Perceptron(random_state=42),
          'LinearSVC': LinearSVC(dual=False, random_state=42),
          'StochasticGradientDescent': SGDClassifier(random_state=42),
          'LinearDiscriminantAnalysis': LinearDiscriminantAnalysis()}

df_model_comparison = model_comparison(models, X_train, y_train)
df_model_comparison

### Filter features

In [ ]:
df.columns

In [ ]:
###drop certain features###

#basis with all features (rfc validation score: 0.83166)
drop_features_round0 = []

#based on high related features in dendogram (rfc validation score: 0.82719)
drop_features_round1 = ['CategoricalFare', 'CategoricalAge', 'IsMarried', 'DeckGroup', 'DeckGroup_M', 'Embarked_S', 'FamilySizeBin', 'FamilySizeBin_1', 'CategoricalAge_(-0.08, 16.0]']

#based on the worst feature determined by the accurate feature importance (rfc validation score: 0.82833)
drop_features_round2 = ['FamilySizeBin_4']

#added the worst feature after refiting the model with drop features of round 2 (rfc validation score: 0.82381)
drop_features_round3 = ['FamilySizeBin_4', 'Person']

#added the worst feature after refiting the model with drop features of round 3 (rfc validation score: 0.82268)
drop_features_round4 = ['FamilySizeBin_4', 'Person', 'Person_child']

#added a feature wich did not change model performace because there were no negative features (rfc validation score: 0.82268)
drop_features_round5 = ['FamilySizeBin_4', 'Person', 'Person_child', 'CategoricalAge_(16.0, 32.0]']

#added the 3 worst features of round 5 to the list because they had the same score (rfc validation score: 0.8283)
drop_features_round6 = ['FamilySizeBin_4', 'Person', 'Person_child', 'CategoricalAge_(16.0, 32.0]', 'Pclass', 'Deck', 'Title_Mrs'] 

#added the last worst feature because after this round there are no negative or zero impact features (rfc validation score: 0.82605)
drop_features_round7 = ['FamilySizeBin_4', 'Person', 'Person_child', 'CategoricalAge_(16.0, 32.0]', 'Pclass', 'Deck', 'Title_Mrs', 'CategoricalFare_1']

###keep certain features###

#best features based on previous feature selection (rfc validation score: 0.83841)
keep_features_round0 = ['Person', 'Sex', 'Person_male', 'Person_female', 'Title_Mr', 'Pclass', 'Title', 'Age', 'Fare', 'Pclass_3',
                        'FamilySize', 'Deck', 'TicketFrequency', 'Title_Mrs', 'FamilySizeBin', 'HasCabin', 'CategoricalFare', 'Title_Miss', 'IsMarried', 'DeckGroup']

#best features based on feature importance of the rfc and svc ensemble model (ensemble validation score: 0.8317)
keep_features_round1 = ['Person', 'Sex', 'Person_male', 'Title_Mr', 'Pclass', 'Title', 'Age', 'Fare', 'Pclass_3',
                        'FamilySize', 'Deck', 'TicketFrequency', 'Title_Mrs', 'FamilySizeBin', 'HasCabin', 'CategoricalFare', 'Title_Miss', 'IsMarried', 'DeckGroup']

#update of best features based on feature importance of the rfc and svc ensemble model (ensemble validation score: 0.81036)
keep_features_round2 = ['Sex', 'Title_Mr', 'Pclass', 'Title', 'Age', 'Fare', 'Pclass_3', 'FamilySize', 'Deck', 'TicketFrequency', 'Title_Mrs', 
                        'FamilySizeBin', 'HasCabin', 'CategoricalFare', 'Title_Miss', 'IsMarried', 'DeckGroup']

In [ ]:
filter_option = 'Keep'                  # choose to drop or keep features
drop_features = drop_features_round0    # which list of features to drop
keep_features = keep_features_round0    # which list of features to keep

if filter_option == 'Drop':

    drop_features_train = ['Survived'] + drop_features
    X_train_filter = df_train.drop(drop_features_train, axis=1).values 
    
    drop_features_test = drop_features
    X_test_filter = df_test.drop(drop_features_test, axis=1).values

    features_included = df_train.drop(drop_features_train, axis=1).columns 

elif filter_option == 'Keep':
    
    X_train_filter = df_train[keep_features].values 
    X_test_filter = df_test[keep_features].values

    features_included = df_train[keep_features].columns

### Optimize model hyperparameters

In [ ]:
X_train_gs = X_train_filter             # complete or filtered dataset
run_gs = False                          # doing a complete grid search  
model_selected = 'lr'                   # choise between rfc, svc, lr for doing a grid search
model_needed = ['rfc', 'svc']           # for which models is fitted estimator needed             
ensembling = True

if run_gs:

    kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

    if model_selected == 'rfc':

        pipe = Pipeline([('scale', StandardScaler()), ('rfc', RandomForestClassifier(random_state=42))])

        params = {
        'rfc__n_estimators': [50, 100, 250, 300, 400, 500, 600, 750],
        'rfc__max_depth': [5, 8, 15, 25, 30, 50, 100],
        'rfc__min_samples_split': [2, 5, 10, 15, 100],
        'rfc__min_samples_leaf': [1, 2, 5, 10],
        'rfc__max_features': ['auto', 'sqrt', 'log2'],
        'rfc__bootstrap': [True, False] 
        }

    elif model_selected == 'svc':

        pipe = Pipeline([('scale', StandardScaler()), ('svc', SVC(random_state=42))])

        params = {
        'svc__kernel': ['linear', 'rbf', 'sigmoid'],
        'svc__gamma': [0.0001, 0.001, 0.01, 1, 10],
        'svc__C': [0.1, 1, 10, 100]
        }

    elif model_selected == 'lr':

        pipe = Pipeline([('scale', StandardScaler()), ('lr', LogisticRegression(random_state=42))])

        params = {
        'lr__penalty': ['l1', 'l2', 'elasticnet', 'none'],
        'lr__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        'lr__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
        'lr__max_iter': list(range(100,1000,100))
        }

    model = GridSearchCV(pipe, params, scoring = 'accuracy', cv = kf, verbose = 2, n_jobs = -1, refit=True)
    
    start = time.time()
    model.fit(X_train_gs, y_train)
    end = time.time()

    print("\n Results from Grid Search")
    print("\n Duration of GridSearch in minutes: ", round((end - start)/60, 2))
    print("\n The best estimator across ALL searched params:\n",model.best_estimator_)
    print("\n The best score across ALL searched params:\n",model.best_score_.round(6))
    print("\n The best parameters across ALL searched params:\n",model.best_params_)

else:

    train_gs_scaler = StandardScaler()
    X_train_gs_scaled = train_gs_scaler.fit_transform(X_train_gs)
    
    estimators=[]

    for models in model_needed:

        if models == 'rfc':
            
            best_params_rfc = {
                'n_estimators': 500,
                'max_depth': 15,
                'min_samples_split': 5,
                'min_samples_leaf': 2,
                'max_features': 'auto',
                'bootstrap': True
            }
            
            best_model_rfc = RandomForestClassifier(random_state=42, **best_params_rfc)
            best_model_rfc.fit(X_train_gs_scaled, y_train)
            estimators.append(("rfc", best_model_rfc))

            print("\n The training score of the best rfc model:\n",best_model_rfc.score(X_train_gs_scaled, y_train).round(5))
            print(" The validation score of the best rfc model:\n",cross_val_score(best_model_rfc, X_train_gs_scaled, y_train, cv=10, scoring='accuracy').mean().round(5))
        
        elif models == 'svc':     

            best_params_svc = {     
                'kernel': 'rbf',
                'gamma': 0.001, 
                'C': 100
            }

            best_model_svc = SVC(random_state=42, **best_params_svc)
            best_model_svc.fit(X_train_gs_scaled, y_train)
            estimators.append(("svc", best_model_svc))

            print("\n The training score of the best svc model:\n",best_model_svc.score(X_train_gs_scaled, y_train).round(5))
            print(" The validation score of the best svc model:\n",cross_val_score(best_model_svc, X_train_gs_scaled, y_train, cv=10, scoring='accuracy').mean().round(5))

        elif models == 'lr':     

            best_params_lr = {     
                'penalty': 'l1',
                'solver': 'saga',
                'C': 0.1,
                'max_iter': 200 #at 100 the coef_ did not converge
            }

            best_model_lr = LogisticRegression(random_state=42, **best_params_lr)
            best_model_lr.fit(X_train_gs_scaled, y_train)
            estimators.append(("lr", best_model_lr))

            print("\n The training score of the best lr model:\n",best_model_lr.score(X_train_gs_scaled, y_train).round(5))
            print(" The validation score of the best lr model:\n",cross_val_score(best_model_lr, X_train_gs_scaled, y_train, cv=10, scoring='accuracy').mean().round(5))

    if ensembling:

        ensemble_model = VotingClassifier(estimators)      
        ensemble_model.fit(X_train_gs_scaled, y_train)
            
        print("\n The training score of the ensemble model:\n",ensemble_model.score(X_train_gs_scaled, y_train).round(5))
        print(" The validation score of the ensemble model:\n",cross_val_score(ensemble_model, X_train_gs_scaled, y_train, cv=10, scoring='accuracy').mean().round(5))

### Check feature importances

In [ ]:
best_model = ensemble_model     # which model to check feature importance
weights = False                 # show weights of model, only possible with certain models
accuracy = True                 # most accuracte option but also more time consuming
use_trees = False               # whether to use the permutation importances option with trees 

if accuracy:

    #save the benchmark score, need to be similar to training score above
    benchmark_score = best_model.score(X_train_gs_scaled, y_train)
    print('The benchmark score is: ', benchmark_score.round(5))

    importances = []

    for idx, item in enumerate(features_included):
        
        #make a clone of the best model
        model_clone = clone(best_model)
        model_clone.random_state = 42

        #drop every time one of the features
        X_train_fi = np.delete(X_train_gs_scaled, idx, axis = 1)

        #refit the model and calculate training score
        model_clone.fit(X_train_fi, y_train)
        drop_col_score = model_clone.score(X_train_fi, y_train)
        importances.append(benchmark_score - drop_col_score)

    #save the data in a dataframe
    accur_import = pd.DataFrame(data={
            'Features': features_included,
            'Bench - Drop': importances
        })
        
    accur_import = accur_import.sort_values(by='Bench - Drop', ascending=False)
    accur_import = accur_import.set_index(accur_import.columns[0])

    #plot the results in a bar graph
    accur_import.plot(kind='barh', figsize=(8, 12), title = 'Difference between benchmark and dropped feature model')
    
else:
    
    if use_trees == True:
        #plot feature importances (only with trees)
        importances = best_model.feature_importances_
        sorted_indices = np.argsort(importances)[::-1]

        plt.figure(figsize=(15, 7))
        plt.title('Feature Importances')
        plt.bar(range(X_train_gs_scaled.shape[1]), importances[sorted_indices], align='center')
        plt.xticks(range(X_train_gs_scaled.shape[1]), features_included[sorted_indices], rotation=90)
        plt.tight_layout()
        plt.show()

    else:

        if weights:
            #plot weights assigned to features, only possible in a few situations
            weights = pd.DataFrame(data={
                'Features': features_included,
                'Weights': best_model.coef_[0]
            })

            weights = weights.sort_values(by='Weights', ascending=False)
            weights = weights.set_index(weights.columns[0])

            weights.plot(kind='bar', figsize=(15, 7), title = 'Feature Weights')

        else:
            #plot permutation feature importance
            permimport = permutation_importance(best_model, X_train_gs_scaled, y_train, n_repeats=10, random_state=42)
            permimport_sorted_idx = permimport.importances_mean.argsort()

            fig, ax = plt.subplots(figsize=(8, 12))
            ax.boxplot(permimport.importances[permimport_sorted_idx].T, vert=False, labels=features_included[permimport_sorted_idx])
            ax.set_title("Permutation Importances")
            plt.show()

In [ ]:
#check for correlated features
fig, ax = plt.subplots(figsize=(15, 7))

corr = spearmanr(X_train_gs_scaled).correlation
corr_linkage = hierarchy.ward(corr)
dendro = hierarchy.dendrogram(corr_linkage, labels=features_included, ax=ax, leaf_rotation=90)

ax.set_title('Dendrogram')
fig.tight_layout()
plt.show()

## Make a submission

In [ ]:
#select the final model 
best_model = ensemble_model

#select which data to use for making predictions
X_test_pred = X_test_filter

#use scaler of training dataset
X_test_scaled = train_gs_scaler.transform(X_test_pred)

#make the predictions
pred_test = best_model.predict(X_test_scaled)

In [ ]:
#check if the model is better as a basic 1 or 0 prediction
display(pred_test.mean())
display(pred_test)

In [ ]:
sub = pd.DataFrame()
sub['PassengerId'] = range(892, 1310, 1)
sub['Survived'] = pred_test

In [ ]:
def savesub (df):
     ts = datetime.now().strftime("%d%m%y_%H%M")
    
     files = next(os.walk("C:/Users/Peter/Documents/Kaggle/Titanic/Submissions/"))[2]
     file_count = len(files)
     
     name_file = 'submission{}_{}.csv'.format(file_count, ts)
     print(name_file)

     name_model = 'model{}_{}.pkl'.format(file_count, ts)
     print(name_model)
     
     #save submission
     df.to_csv(r'C:/Users/Peter/Documents/Kaggle/Titanic/Submissions/' + name_file, index=False)

     #save model
     joblib.dump(best_model, r'C:/Users/Peter/Documents/Kaggle/Titanic/Models/' + name_model)

In [ ]:
savesub(sub)